In [1]:
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load and preprocess data from Excel
excel_path = '/content/beam_prediction_dataset.xlsx'  # Change this to your file path
data = pd.read_excel(excel_path)

ERROR:jax._src.xla_bridge:Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.10/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


In [9]:
import tensorflow as tf
import tensorflow_federated as tff

# Step 1: Define the Keras model
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='sgd', loss='mean_squared_error')
    return model

# Step 2: Define the model function
def model_fn():
    keras_model = create_keras_model()

    # TFF model must have forward pass
    def forward_pass(model, batch):
        x = batch[0]
        y = batch[1]
        predictions = model(x)
        loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(y, predictions))
        return tff.learning.BatchOutput(loss=loss, predictions=predictions)

    return forward_pass(keras_model, (tf.random.uniform([32, 5]), tf.random.uniform([32, 1]))) # Dummy data for example

# Step 3: Create Federated Averaging process manually
def federated_averaging_fn(model, client_optimizer, server_optimizer):
    model_weights = model.trainable_variables
    client_state = tf.nest.map_structure(lambda x: tf.zeros_like(x), model_weights)

    def client_update(model, dataset):
        # Simulate the dataset as a tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices(dataset).batch(32)

        # Perform training using the dataset (features, labels)
        for batch in dataset:
            x, y = batch
            with tf.GradientTape() as tape:
                predictions = model(x)
                loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(y, predictions))
            grads = tape.gradient(loss, model.trainable_variables)
            client_optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return model.trainable_variables

    def server_update(model_weights, aggregated_weights):
        new_weights = [
            (1.0 / len(aggregated_weights)) * sum([w for w in aggregated_weights])
            for aggregated_weights in zip(*aggregated_weights)
        ]
        model.set_weights(new_weights)
        return model.trainable_variables

    def federated_process(state, federated_data):
        client_weights = [client_update(model, client_data) for client_data in federated_data]
        updated_weights = server_update(model_weights, client_weights)
        return updated_weights, model_weights

    return federated_process

# Step 4: Simulate federated data
federated_train_data = [(tf.random.uniform([100, 5]), tf.random.uniform([100, 1]))] * 10  # Simulating 10 clients

# Optimizers
client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer = tf.keras.optimizers.SGD(learning_rate=1.0)

# Define the model (using Keras directly in the model function)
model = create_keras_model()

# Define federated learning process
federated_process = federated_averaging_fn(model, client_optimizer, server_optimizer)

# Step 5: Execute federated learning for 10 rounds
for round_num in range(10):
    # Ensure the federated data is passed as a list of (features, labels) pairs for each client
    federated_data = [(tf.random.uniform([100, 5]), tf.random.uniform([100, 1])) for _ in range(10)]
    state = federated_process(model.trainable_variables, federated_data)
    print(f"Round {round_num} completed.")


Round 0 completed.
Round 1 completed.
Round 2 completed.
Round 3 completed.
Round 4 completed.
Round 5 completed.
Round 6 completed.
Round 7 completed.
Round 8 completed.
Round 9 completed.


In [10]:
def evaluate_model(model, federated_data):
    total_loss = 0
    num_clients = len(federated_data)

    # Evaluate the model on each client's data
    for client_data in federated_data:
        dataset = tf.data.Dataset.from_tensor_slices(client_data).batch(32)
        for batch in dataset:
            x, y = batch
            predictions = model(x)
            loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(y, predictions))
            total_loss += loss

    # Compute average loss over all clients
    average_loss = total_loss / num_clients
    return average_loss

# After training each round, evaluate the model:
for round_num in range(10):
    state = federated_process(model.trainable_variables, federated_data)
    print(f"Round {round_num} completed.")

    # Evaluate after each round
    avg_loss = evaluate_model(model, federated_data)
    print(f"Average loss after round {round_num}: {avg_loss}")


Round 0 completed.
Average loss after round 0: 0.3264043927192688
Round 1 completed.
Average loss after round 1: 0.32531046867370605
Round 2 completed.
Average loss after round 2: 0.32438474893569946
Round 3 completed.
Average loss after round 3: 0.3235413432121277
Round 4 completed.
Average loss after round 4: 0.322805255651474
Round 5 completed.
Average loss after round 5: 0.32214072346687317
Round 6 completed.
Average loss after round 6: 0.32157057523727417
Round 7 completed.
Average loss after round 7: 0.3210555911064148
Round 8 completed.
Average loss after round 8: 0.3205874264240265
Round 9 completed.
Average loss after round 9: 0.32013753056526184


HYPERPARAMETER TUNING


In [11]:
from sklearn.model_selection import ParameterGrid
import tensorflow as tf

# Hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128]
}

# Example function for training the model with federated learning
def train_with_hyperparameters(learning_rate, batch_size):
    model = create_keras_model()
    client_optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    server_optimizer = tf.keras.optimizers.SGD(learning_rate=1.0)

    federated_process = federated_averaging_fn(model, client_optimizer, server_optimizer)

    federated_train_data = [(tf.random.uniform([batch_size, 5]), tf.random.uniform([batch_size, 1]))] * 10  # Simulating 10 clients

    # Execute federated learning for a few rounds
    for round_num in range(10):
        federated_data = [(tf.random.uniform([batch_size, 5]), tf.random.uniform([batch_size, 1])) for _ in range(10)]
        federated_process(model.trainable_variables, federated_data)

    return model  # Return the model after training

# Grid search for hyperparameter tuning
best_model = None
best_loss = float('inf')

for params in ParameterGrid(param_grid):
    print(f"Testing with params: {params}")
    model = train_with_hyperparameters(params['learning_rate'], params['batch_size'])
    # Evaluate the model (you can define a proper evaluation function based on your problem)
    loss = evaluate_model(model, federated_train_data)

    if loss < best_loss:
        best_loss = loss
        best_model = model

print(f"Best model achieved with loss: {best_loss}")


Testing with params: {'batch_size': 32, 'learning_rate': 0.001}
Testing with params: {'batch_size': 32, 'learning_rate': 0.01}
Testing with params: {'batch_size': 32, 'learning_rate': 0.1}
Testing with params: {'batch_size': 64, 'learning_rate': 0.001}
Testing with params: {'batch_size': 64, 'learning_rate': 0.01}
Testing with params: {'batch_size': 64, 'learning_rate': 0.1}
Testing with params: {'batch_size': 128, 'learning_rate': 0.001}
Testing with params: {'batch_size': 128, 'learning_rate': 0.01}
Testing with params: {'batch_size': 128, 'learning_rate': 0.1}
Best model achieved with loss: 0.3558220863342285


MODEL AGGREGATION


In [12]:
# Model aggregation function
def federated_averaging_fn(model, client_optimizer, server_optimizer):
    model_weights = model.trainable_variables
    client_state = tf.nest.map_structure(lambda x: tf.zeros_like(x), model_weights)

    def client_update(model, dataset):
        # Simulate the dataset as a tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices(dataset).batch(32)

        for batch in dataset:
            x, y = batch
            with tf.GradientTape() as tape:
                predictions = model(x)
                loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(y, predictions))
            grads = tape.gradient(loss, model.trainable_variables)
            client_optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return model.trainable_variables

    def server_update(model_weights, aggregated_weights):
        new_weights = [
            (1.0 / len(aggregated_weights)) * sum([w for w in aggregated_weights])
            for aggregated_weights in zip(*aggregated_weights)
        ]
        model.set_weights(new_weights)
        return model.trainable_variables

    def federated_process(state, federated_data):
        client_weights = [client_update(model, client_data) for client_data in federated_data]
        updated_weights = server_update(model_weights, client_weights)
        return updated_weights, model_weights

    return federated_process


 Testing and Deployment

In [13]:
# Test the model
def evaluate_model(model, federated_data):
    # Assuming federated_data contains the test data for evaluation
    total_loss = 0
    for data in federated_data:
        x, y = data
        predictions = model(x)
        loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(y, predictions))
        total_loss += loss

    avg_loss = total_loss / len(federated_data)
    return avg_loss

# Deployment example
def deploy_model(model):
    # Here we save the model and assume a simple deployment process
    model.save("federated_model.h5")
    print("Model saved for deployment")

# After training, test the model
test_data = [(tf.random.uniform([100, 5]), tf.random.uniform([100, 1]))] * 10  # Simulated test data
avg_loss = evaluate_model(best_model, test_data)
print(f"Final evaluation loss: {avg_loss}")

# Deploy the model
deploy_model(best_model)


Final evaluation loss: 0.08864980936050415
Model saved for deployment


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Scaling Federated Learning

In [15]:
import tensorflow as tf

# Updated client update function
def client_update(model, dataset, client_optimizer):
    # Simulate the dataset as a tf.data.Dataset
    dataset = tf.data.Dataset.from_tensor_slices(dataset).batch(32)

    # Ensure the optimizer is built with model variables
    client_optimizer.build(model.trainable_variables)

    for batch in dataset:
        x, y = batch
        with tf.GradientTape() as tape:
            predictions = model(x)
            loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(y, predictions))
        grads = tape.gradient(loss, model.trainable_variables)
        client_optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return model.trainable_variables


# Model Aggregation (Federated Averaging)
def federated_averaging_fn(model, client_optimizer, server_optimizer):
    model_weights = model.trainable_variables
    client_state = tf.nest.map_structure(lambda x: tf.zeros_like(x), model_weights)

    def client_update_fn(model, federated_data):
        # Here we run client_update for each dataset from different clients
        return [client_update(model, data, client_optimizer) for data in federated_data]

    def server_update_fn(model_weights, client_weights):
        # Averaging model weights
        averaged_weights = [
            tf.reduce_mean([client_weight[i] for client_weight in client_weights], axis=0)
            for i in range(len(client_weights[0]))
        ]
        model.set_weights(averaged_weights)
        return model.trainable_variables

    def federated_process(state, federated_data):
        client_weights = client_update_fn(model, federated_data)
        updated_weights = server_update_fn(model_weights, client_weights)
        return updated_weights, model_weights

    return federated_process


In [16]:
# Simulate large-scale federated learning
def simulate_large_scale_federated_learning(num_clients=100):
    federated_train_data = [(tf.random.uniform([32, 5]), tf.random.uniform([32, 1]))] * num_clients  # Simulated 100 clients

    # Using the previously defined federated_averaging_fn
    client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    server_optimizer = tf.keras.optimizers.SGD(learning_rate=1.0)
    federated_process = federated_averaging_fn(best_model, client_optimizer, server_optimizer)

    # Execute federated learning for more clients (scaling up)
    for round_num in range(10):
        federated_data = [(tf.random.uniform([32, 5]), tf.random.uniform([32, 1])) for _ in range(num_clients)]
        federated_process(best_model.trainable_variables, federated_data)

    print(f"Federated Learning completed for {num_clients} clients")

# Simulate large-scale federated learning with 100 clients
simulate_large_scale_federated_learning(num_clients=100)


Federated Learning completed for 100 clients
